In [2]:
# import os
# os.system('source activate py35milsed')

0

In [32]:
%matplotlib inline
import keras as K
import pickle
import os
import milsed
import pandas as pd
from sklearn.model_selection import ShuffleSplit

In [10]:
keras.__version__

NameError: name 'keras' is not defined

In [25]:
def construct_model(pump, alpha):

    model_inputs = ['mel/mag']

    # Build the input layer
    layers = pump.layers()

    x_mag = layers['mel/mag']

    # Apply batch normalization
    x_bn = K.layers.BatchNormalization()(x_mag)

    x_sq = milsed.layers.SqueezeLayer()(x_bn)

    # First convolutional filter: a single 3-frame filters
    conv1 = K.layers.Convolution1D(64, 3,
                                   padding='same',
                                   activation='relu',
                                   kernel_initializer='he_uniform')(x_sq)
                                   # data_format='channels_last')

    # First recurrent layer: a 128-dim bidirectional gru
    rnn1 = K.layers.Bidirectional(K.layers.GRU(128,
                                               return_sequences=True))(conv1)

    n_classes = pump.fields['static/tags'].shape[0]

    p0 = K.layers.Dense(n_classes, activation='sigmoid')

    p_dynamic = K.layers.TimeDistributed(p0, name='dynamic/tags')(rnn1)

    p_static = milsed.layers.SoftMaxPool(alpha=alpha,
                                         axis=1,
                                         name='static/tags')(p_dynamic)

    model = K.models.Model([x_mag],
                           [p_dynamic, p_static])

    model_outputs = ['dynamic/tags', 'static/tags']

    return model, model_inputs, model_outputs

In [26]:
pump = pickle.load(open('/home/js7561/dev/milsed/models/resources/pump.pkl', 'rb'))

In [27]:
model, model_inputs, model_outputs = construct_model(pump, alpha=1.0)

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mel/mag (InputLayer)         (None, None, 128, 1)      0         
_________________________________________________________________
batch_normalization_5 (Batch (None, None, 128, 1)      4         
_________________________________________________________________
squeeze_layer_4 (SqueezeLaye (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          24640     
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 256)         148224    
_________________________________________________________________
dynamic/tags (TimeDistribute (None, None, 17)          4369      
_________________________________________________________________
static/tags (SoftMaxPool)    (None, 17)                0         
Total para

In [18]:
layers = pump.layers()
x_mag = layers['mel/mag']


In [19]:
x_mag

<tf.Tensor 'mel/mag_4:0' shape=(?, ?, 128, 1) dtype=float32>

In [21]:
K.backend.image_data_format()

'channels_last'

In [43]:
OUTPUT_PATH = '/home/js7561/dev/milsed/models/resources/'
idx_train_ = pd.read_json(os.path.join(OUTPUT_PATH, 'index_train.json'))

In [44]:
idx_train_

,id
0,YOrfyRpXYWEk_0.000_10.000
1,Yx-j5MdM1KR0_14.000_24.000
2,YnsXP-wWPrLo_30.000_40.000
3,YJ4U0JaCy_Qk_30.000_40.000
4,YfAxHss0GwVs_30.000_40.000
5,Y58tSAz2TItg_30.000_40.000
6,YB-B6Mx2S7E4_8.000_18.000
7,Yzbh1fupFwRM_0.000_10.000
8,YPZrc_ddxLrM_0.000_7.000
9,Ybo_KcZvGHJw_30.000_40.000


In [45]:
splitter_tv = ShuffleSplit(n_splits=1, test_size=0.25,
                               random_state=123)

In [46]:
train, val = next(splitter_tv.split(idx_train_))

In [47]:
train

array([46072, 33497, 27659, ..., 17730, 28030, 15725])

In [48]:
val

array([13216, 20487, 42373, ..., 36100, 43242, 29230])

In [49]:
idx_train = idx_train_.iloc[train]

In [50]:
idx_train

,id
46072,Yn_iUcraHNII_0.000_10.000
33497,YiRy6FxnuIOc_250.000_260.000
27659,YAHbHcjKb4a4_30.000_40.000
34487,YsnZ89jW7zGQ_1.000_11.000
11816,YLtIA5H9cuKQ_21.000_31.000
31949,YIfE6z7nlTJg_50.000_60.000
6493,Y1S6Ajg-X480_21.000_31.000
21226,YN80dCIR8pgg_20.000_30.000
33119,YS6HRDChdC_w_30.000_40.000
35785,YdU7iQl6G7a4_560.000_570.000


In [51]:
idx_val = idx_train_.iloc[val]

In [52]:
idx_val

,id
13216,Y16gysIPq3G8_19.000_29.000
20487,YOKbY-C6Dwl0_10.000_20.000
42373,Ywe197L37cPg_40.000_50.000
42559,YvafWHxgv824_0.000_10.000
41599,YL9gOhEvn4lc_90.000_100.000
186,Y1L1Kcgur6gI_50.000_60.000
21483,YACWJA8d0USg_4.000_14.000
49803,YljNOwoQLnCc_0.000_10.000
4914,Y20BGawT1jo8_30.000_40.000
17746,YmYaKfPjrZps_30.000_40.000


In [53]:
idx_train['id'].values

array(['Yn_iUcraHNII_0.000_10.000', 'YiRy6FxnuIOc_250.000_260.000',
       'YAHbHcjKb4a4_30.000_40.000', ..., 'Yhc8KZ3WpEi8_30.000_40.000',
       'YQV1MALb5lp0_30.000_40.000', 'YCO1BTFTJRUg_0.000_9.000'], dtype=object)